# Sustainable Sourcing Layers 2025a visualization

This notebook demonstrates how to access, analyze and visualize datasets developed by Google, FAO and others in support of the [Forest Data Partnership](https://www.forestdatapartnership.org/).  In particular, explore the 2025a release of Forest Data Partnership sustainable sourcing layers including palm, rubber, cocoa and coffee.  Demonstrations below include:

- Visualize probability layers.
- Visualize data coverage (masks).
- Visualize a mini-ensemble of thresholded probabiliites.
- Get statistics in image regions from the Earth Engine Python client.

For details on how these layers were produced, see [the technical documentation on GitHub](https://github.com/google/forest-data-partnership/tree/main/models).  In particular, see [the limitations](https://github.com/google/forest-data-partnership/tree/main/models#limitations).  See also the [Forest Data Partnership publisher catalog](https://developers.google.com/earth-engine/datasets/publisher/forestdatapartnership) for dataset descriptions.  See [this Earth Engine Code Editor script](https://goo.gle/fodapa-layers) for a demonstration of how choice of thresholds affects the mapped results.

Note that users of commercial projects will need to request access through [this form](https://docs.google.com/forms/d/e/1FAIpQLSe7L3eh6t2JIPqEtAQwXwY7ZmW52v8W5vrIi4QN_XYgTNJZLw/viewform).


**WARNING**: These demos consume billable resources and may result in charges to your account!

# Setup and Auth

Import the required libraries.

In [2]:
import ee
import geemap
from google.api_core import exceptions, retry
import google.auth
from google.colab import auth, userdata

print('Using EE version: ', ee.__version__)
print('Using geemap version: ', geemap.__version__)

Using EE version:  1.5.24
Using geemap version:  0.35.3


Load the GCP project and compute region information from Colab secrets.
See [this guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for information on how to use Colab secrets.

In [8]:
PROJECT = userdata.get('EE_PROJECT_ID')
REGION = userdata.get('GCP_REGION')

Authenticate and initialize Earth Engine.

In [9]:
auth.authenticate_user()

credentials, _ = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/earthengine",
    ]
)
ee.Initialize(
    credentials,
    project=PROJECT,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

# Define Data Layers

## Forest Data Partnership Sustainable Sourcing Layers

See the [Forest Data Partnership publisher catalog](https://developers.google.com/earth-engine/datasets/publisher/forestdatapartnership) for dataset descriptions.

In [10]:
cocoa = ee.ImageCollection("projects/forestdatapartnership/assets/cocoa/model_2025a")
coffee = ee.ImageCollection("projects/forestdatapartnership/assets/coffee/model_2025a")
palm = ee.ImageCollection("projects/forestdatapartnership/assets/palm/model_2025a")
rubber = ee.ImageCollection("projects/forestdatapartnership/assets/rubber/model_2025a")

## Google Deep Mind Natural Forest

See [Neumann et al. 2025](https://eartharxiv.org/repository/view/9085/) for details.

In [12]:
nf = ee.ImageCollection(
  'projects/computing-engine-190414/assets/biosphere_models/public/forest_typology/natural_forest_2020_v1_0')
nf_image = nf.mosaic().divide(255).selfMask()

## Visualize with geemap

Note that the probability layers are visualized using default values.  The data masks show coverage, or where data are available.  Toggle the display of these layers in the `geemp` `Map`.  See [the `geemap` docs](https://geemap.org/) for more information.

In [13]:
# Center over Indonesia.
Map = geemap.Map(center=(-3.416, 104.3318), zoom=9)

In [14]:
# Filter the datasets by date and add to the Map.
filter2020 = ee.Filter.calendarRange(2020, 2020, 'year')
filter2023 = ee.Filter.calendarRange(2023, 2023, 'year')

cocoa2020 = cocoa.filter(filter2020).mosaic()
Map.addLayer(cocoa2020.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'blue'},
             'cocoa 2020',
             False)

cocoa2023 = cocoa.filter(filter2023).mosaic()
Map.addLayer(cocoa2023.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'blue'},
             'cocoa 2023',
             False)

coffee2020 = coffee.filter(filter2020).mosaic()
Map.addLayer(coffee2020.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'yellow'},
             'coffee 2020',
             False)

coffee2023 = coffee.filter(filter2023).mosaic()
Map.addLayer(coffee2023.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'yellow'},
             'coffee 2023',
             False)

palm2020 = palm.filter(filter2020).mosaic()
Map.addLayer(palm2020.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'purple'},
             'palm 2020',
             False)

palm2023 = palm.filter(filter2023).mosaic()
Map.addLayer(palm2023.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'purple'},
             'palm 2023',
             False)

rubber2020 = rubber.filter(filter2020).mosaic()
Map.addLayer(rubber2020.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'red'},
             'rubber 2020',
             False)

rubber2023 = rubber.filter(filter2023).mosaic()
Map.addLayer(rubber2023.selfMask(),
             {'min': 0.5, 'max': 1, 'palette': 'red'},
             'rubber 2023',
             False)

# Coverage, or where there are data for a given year.
p2023mask = palm2023.mask()
r2023mask = rubber2023.mask()
c2023mask = cocoa2023.mask()
j2023mask = coffee2023.mask()
Map.addLayer(p2023mask.selfMask(),
             {'palette': 'purple'},
             'palm 2023 coverage',
             False)
Map.addLayer(r2023mask.selfMask(),
             {'palette': 'red'},
             'rubber 2023 coverage',
             False)
Map.addLayer(c2023mask.selfMask(),
             {'palette': 'blue'},
             'cocoa 2023 coverage',
             False)
Map.addLayer(j2023mask.selfMask(),
             {'palette': 'yellow'},
             'coffee 2023 coverage',
             False)

# Intersection of the coverage masks in 2020.
intersection_2020 = ee.ImageCollection.fromImages(
      [palm2020.mask(), rubber2020.mask(), cocoa2020.mask(), coffee2020.mask()]
    ).min()
Map.addLayer(intersection_2020,
             {},
             '2020 Coverage',
             False)

In [15]:
Map

Map(center=[-3.416, 104.3318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

## Create a mini-ensemble

Apply some thresholds to the data.  The values chosen here **are for demonstration purposes only** and should be tuned for any particular geographic region or use case.  See [the technical documentation on GitHub](https://github.com/google/forest-data-partnership/tree/main/models) for details on the estimated accuracy at different thresholds.  See [this Earth Engine Code Editor script](https://goo.gle/fodapa-layers) for a demonstration of how choice of thresholds affects the mapped results.

In [ ]:
# Arbitrary thresholds.  Tune these to your needs!
thresholds = [
    0.5,
    0.45,
    0.96,
    0.89,
    0.5
  ]

# A mini-ensemble of GDM and fodapa data products.
ensemble = ee.Image.cat(
  nf_image.rename('forest'),
  cocoa2020.rename('cocoa'),
  coffee2020.rename('coffee'),
  palm2020.rename('palm'),
  rubber2020.rename('rubber')
).unmask(0)


# Threshold the probabilities.
crop_names = ['forest', 'cocoa', 'coffee', 'palm', 'rubber']
thresholded = ensemble.select(crop_names).gt(ee.Image(thresholds))

# Unclassified means no predicted presence at the specified thresholds.
unclassified = thresholded.reduce('sum').eq(0)

classVis = {
  'min': 0,
  'max': 4,
  'palette': ['green', 'blue', 'yellow', 'purple', 'red']
}
label = (
    thresholded
    .toArray()
    .arrayArgmax()
    .arrayGet(0)
    .rename('classIndex')
    .updateMask(unclassified.Not()))

# Confusion means two or more classes predicted presence.
confusion = thresholded.reduce('sum').gt(1).selfMask()

# Water mask from WorldCover
worldcover = ee.ImageCollection('ESA/WorldCover/v200')
wc_water_class = ee.Image(80)
water = (
      worldcover.mosaic()
      .unmask(wc_water_class, sameFootprint=False)
      .eq(wc_water_class)
      .selfMask()
)

# Create a mosaic to display the mini-ensemble.
ensemble_map = ee.ImageCollection.fromImages([
  unclassified.selfMask().visualize(palette='gray'),
  label.visualize(**classVis),
  confusion.visualize(palette='white'),
  water.visualize(palette='navy')
]).mosaic()

In [ ]:
# Center over Indonesia.
Map2 = geemap.Map(center=(-3.416, 104.3318), zoom=9)
gini = ee.Image(1).subtract(ensemble.multiply(ensemble).reduce('sum'))
Map2.addLayer(gini, {}, 'gini')
Map2.addLayer(ensemble_map, {}, 'Ensemble Maps')

In [ ]:
Map2

## Statistics of image regions

### Load WHISP example data

Here we will get the WHISP example data from GitHub and convert to an `ee.FeatureCollection`.

In [ ]:
import json

fc_list = !curl https://raw.githubusercontent.com/forestdatapartnership/whisp/main/tests/fixtures/geojson_example.geojson
fc_obj = json.loads("\n".join(fc_list))
fc = ee.FeatureCollection(fc_obj)

### Get area stats using Earth Engine

Compute [Gini impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) on the result, a standard index for understanding node impurity in descision tree contstruction.

In [ ]:
def get_stats(feature):
  """Add commodity area stats to a feature.  For demonstration only!"""
  feature_area = ee.Number(feature.geometry().area(10))

  stats = ee.Image.cat(
      thresholded,
      confusion.rename('confusion'),
      unclassified.rename('unclassified')
  ).multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=feature.geometry(),
    scale=10,
  )

  gini = ee.Number(1).subtract(ee.List(
      [ee.Number(stats.get(c)).divide(feature_area) for c in crop_names]
  ).reduce(ee.Reducer.sum()))

  stats = stats.set({'gini': gini, 'total_area': feature_area})

  # Get rid of other attributes.
  return feature.select([]).set(stats)

In [ ]:
pprint(fc.map(get_stats).getInfo())

## Next steps

- Check the [Suso data Cloud Function demo notebook](https://colab.research.google.com/drive/1cQyqNaiK3nP65I-LunRkQyLLXTakaYs9?resourcekey=0-YAkoE8VC9drgqa1PE6uAGA&usp=sharing)
- Check the [WHISP Cloud Function demo notebook](https://colab.research.google.com/drive/1NCaPOoxqmAEWb8c8V0kEHVunbW1yHVkL?resourcekey=0-HJ3ou94AbjdKkkvaPW1Jtw&usp=sharing)